In [1]:
# Setting the basic parameters
T = 12 # Using 12 months to forecast the 13th month
I = 6 # Invetsment cycle
K = 6 # Optimal number of factors provided by the literature
N = 4000 # Using part of the valid stocks
S = 30 # Actual investment willingness
long_margin = 100
short_margin = 110
industry = ['能源', '石油', '发电'] # The key words of the industry selected (e.g. '能源', '石油', '发电'), [] if no industry is selected
method = 2 # Alpha extraction method used in the estimation, 1 for the first method provided by the literature, 2 for the second

financial_metrics_list = [
    "ROE",
    "ROA",
    "ROIC",
    "Gross profit to revenue",
    "Operating profit to revenue",
    "Operating cost to revenue",
    "EBITDA to revenue",
    "NI to revenue",
    "Net operating CF to revenue",
    "CapEx to depreciation",
    "Current asset ratio",
    "Asset liability ratio",
    "Current liability ratio",
    "Tangible asset ratio",
    "EPS basic",
    "EPS diluted",
    "Net asset per share",
    "Net operating CF per share",
    "Revenue per share",
    "Capital reserve per share",
    "Retained earnings per share",
    "FCF per share",
    "Equity multiplier"
]

In [2]:
import pandas as pd
import numpy as np
import datetime, calendar
from scipy.sparse.linalg import eigsh

In [3]:
# Loading data
panel = pd.read_csv('characteristics-2024-02-01.csv', index_col=[0,1], low_memory=False).apply(pd.to_numeric, errors='coerce')
panel.index = pd.MultiIndex.from_arrays([panel.index.get_level_values(0), pd.to_datetime(panel.index.get_level_values(1))])
panel

Alpha  Asset liability ratio      Beta  \
Ticker Date                                                    
AAMC.A 2016-03-31 -2.180497                    NaN  2.079191   
       2016-04-30  1.836316                    NaN  2.704243   
       2016-05-31  1.182486                    NaN -0.143233   
       2016-06-30 -0.758921                    NaN  2.846175   
       2016-07-31 -1.060616                    NaN  3.023535   
...                     ...                    ...       ...   
ZYXI.O 2023-09-30  0.668884                61.3139  1.850362   
       2023-10-31  0.680420                61.3139  1.187454   
       2023-11-30 -0.386956                61.3139  1.700408   
       2023-12-31  0.466776                61.3139  2.020288   
       2024-01-31  0.330758                61.3139  1.312379   

                   CapEx to depreciation  Capital reserve per share  \
Ticker Date                                                           
AAMC.A 2016-03-31                    NaN                        NaN   
       2016-04-30                    NaN                        NaN   
       2016-05-31                    NaN                        NaN   
       2016-06-30                    NaN                        NaN   
       2016-07-31                    NaN                        NaN   
...                                  ...                        ...   
ZYXI.O 2023-09-30               -0.31754                     2.6458   
       2023-10-31               -0.31754                     2.6458   
       2023-11-30               -0.31754                     2.6458   
       2023-12-31               -0.31754                     2.6458   
       2024-01-31               -0.31754                     2.6458   

                   Current asset ratio  Current liability ratio  \
Ticker Date                                                       
AAMC.A 2016-03-31                  NaN                      NaN   
       2016-04-30                  NaN                      NaN   
       2016-05-31                  NaN                      NaN   
       2016-06-30                  NaN                      NaN   
       2016-07-31                  NaN                      NaN   
...                                ...                      ...   
ZYXI.O 2023-09-30              67.9678                  21.3717   
       2023-10-31              67.9678                  21.3717   
       2023-11-30              67.9678                  21.3717   
       2023-12-31              67.9678                  21.3717   
       2024-01-31              67.9678                  21.3717   

                   EBITDA to revenue  EPS basic  EPS diluted  ...    PS TTM  \
Ticker Date                                                   ...             
AAMC.A 2016-03-31                NaN        NaN          NaN  ...       NaN   
       2016-04-30                NaN        NaN          NaN  ...       NaN   
       2016-05-31                NaN        NaN          NaN  ...       NaN   
       2016-06-30                NaN        NaN          NaN  ...       NaN   
       2016-07-31                NaN        NaN          NaN  ...       NaN   
...                              ...        ...          ...  ...       ...   
ZYXI.O 2023-09-30           9.749192       0.24         0.23  ...  1.473115   
       2023-10-31           9.749192       0.24         0.23  ...  1.620008   
       2023-11-30           9.749192       0.24         0.23  ...  1.671089   
       2023-12-31           9.749192       0.24         0.23  ...  1.986699   
       2024-01-31           9.749192       0.24         0.23  ...  2.160011   

                      ROA      ROE      ROIC  Retained earnings per share  \
Ticker Date                                                                 
AAMC.A 2016-03-31     NaN      NaN       NaN                          NaN   
       2016-04-30     NaN      NaN       NaN                          NaN   
       2016-05-31     NaN      NaN       NaN             

In [4]:
# Column names of the characteristics and time
characteristics = panel.columns.drop('Return')
# Convert date values to Timestamp
dates = panel.index.get_level_values(1).unique()
# Fill any N.A. with previous observations except return (considering the delisting circumstance)
panel_fill = panel.copy()
panel_fill[characteristics] = panel_fill[characteristics].groupby(level=0).transform(lambda group: group.ffill())
panel_fill

Alpha  Asset liability ratio      Beta  \
Ticker Date                                                    
AAMC.A 2016-03-31 -2.180497                    NaN  2.079191   
       2016-04-30  1.836316                    NaN  2.704243   
       2016-05-31  1.182486                    NaN -0.143233   
       2016-06-30 -0.758921                    NaN  2.846175   
       2016-07-31 -1.060616                    NaN  3.023535   
...                     ...                    ...       ...   
ZYXI.O 2023-09-30  0.668884                61.3139  1.850362   
       2023-10-31  0.680420                61.3139  1.187454   
       2023-11-30 -0.386956                61.3139  1.700408   
       2023-12-31  0.466776                61.3139  2.020288   
       2024-01-31  0.330758                61.3139  1.312379   

                   CapEx to depreciation  Capital reserve per share  \
Ticker Date                                                           
AAMC.A 2016-03-31                    NaN                        NaN   
       2016-04-30                    NaN                        NaN   
       2016-05-31                    NaN                        NaN   
       2016-06-30                    NaN                        NaN   
       2016-07-31                    NaN                        NaN   
...                                  ...                        ...   
ZYXI.O 2023-09-30               -0.31754                     2.6458   
       2023-10-31               -0.31754                     2.6458   
       2023-11-30               -0.31754                     2.6458   
       2023-12-31               -0.31754                     2.6458   
       2024-01-31               -0.31754                     2.6458   

                   Current asset ratio  Current liability ratio  \
Ticker Date                                                       
AAMC.A 2016-03-31                  NaN                      NaN   
       2016-04-30                  NaN                      NaN   
       2016-05-31                  NaN                      NaN   
       2016-06-30                  NaN                      NaN   
       2016-07-31                  NaN                      NaN   
...                                ...                      ...   
ZYXI.O 2023-09-30              67.9678                  21.3717   
       2023-10-31              67.9678                  21.3717   
       2023-11-30              67.9678                  21.3717   
       2023-12-31              67.9678                  21.3717   
       2024-01-31              67.9678                  21.3717   

                   EBITDA to revenue  EPS basic  EPS diluted  ...    PS TTM  \
Ticker Date                                                   ...             
AAMC.A 2016-03-31                NaN        NaN          NaN  ...       NaN   
       2016-04-30                NaN        NaN          NaN  ...       NaN   
       2016-05-31                NaN        NaN          NaN  ...       NaN   
       2016-06-30                NaN        NaN          NaN  ...       NaN   
       2016-07-31                NaN        NaN          NaN  ...       NaN   
...                              ...        ...          ...  ...       ...   
ZYXI.O 2023-09-30           9.749192       0.24         0.23  ...  1.473115   
       2023-10-31           9.749192       0.24         0.23  ...  1.620008   
       2023-11-30           9.749192       0.24         0.23  ...  1.671089   
       2023-12-31           9.749192       0.24         0.23  ...  1.986699   
       2024-01-31           9.749192       0.24         0.23  ...  2.160011   

                      ROA      ROE      ROIC  Retained earnings per share  \
Ticker Date                                                                 
AAMC.A 2016-03-31     NaN      NaN       NaN                          NaN   
       2016-04-30     NaN      NaN       NaN                          NaN   
       2016-05-31     NaN      NaN       NaN             

In [5]:
# Only keep the panel needed for current month and fill the NaNs linearly
period = panel_fill[panel_fill.index.get_level_values(1).isin(dates[-1-T:])]

# Data cleaning
na_tickers = period[period.isna().any(axis=1)].index.get_level_values(0).unique()
period = period.drop(na_tickers, axis=0, level=0)

# Selection stocks based on alpha
period['Alpha_Average'] = period['Alpha'].groupby('Ticker').transform('mean')
top_alpha_tickers = period.sort_values(by='Alpha_Average', ascending=False).index.get_level_values(0).unique()[:N]
period = period[period.index.get_level_values(0).isin(top_alpha_tickers)].drop('Alpha_Average', axis=1)
tickers = period.index.get_level_values(0).unique()

period

Alpha  Asset liability ratio      Beta  \
Ticker Date                                                    
ACU.A  2023-01-31  0.224970                51.9214  0.572586   
       2023-02-28  0.281927                51.9214  0.541587   
       2023-03-31 -0.297117                49.1224  0.405839   
       2023-04-30  0.762540                49.1224 -0.171431   
       2023-05-31 -0.198533                49.1224  0.557062   
...                     ...                    ...       ...   
ZYXI.O 2023-09-30  0.668884                61.3139  1.850362   
       2023-10-31  0.680420                61.3139  1.187454   
       2023-11-30 -0.386956                61.3139  1.700408   
       2023-12-31  0.466776                61.3139  2.020288   
       2024-01-31  0.330758                61.3139  1.312379   

                   CapEx to depreciation  Capital reserve per share  \
Ticker Date                                                           
ACU.A  2023-01-31              -1.648840                     3.8008   
       2023-02-28              -1.648840                     3.8008   
       2023-03-31              -0.961591                     3.9242   
       2023-04-30              -0.961591                     3.9242   
       2023-05-31              -0.961591                     3.9242   
...                                  ...                        ...   
ZYXI.O 2023-09-30              -0.317540                     2.6458   
       2023-10-31              -0.317540                     2.6458   
       2023-11-30              -0.317540                     2.6458   
       2023-12-31              -0.317540                     2.6458   
       2024-01-31              -0.317540                     2.6458   

                   Current asset ratio  Current liability ratio  \
Ticker Date                                                       
ACU.A  2023-01-31              64.2425                  25.9255   
       2023-02-28              64.2425                  25.9255   
       2023-03-31              62.9868                  31.0697   
       2023-04-30              62.9868                  31.0697   
       2023-05-31              62.9868                  31.0697   
...                                ...                      ...   
ZYXI.O 2023-09-30              67.9678                  21.3717   
       2023-10-31              67.9678                  21.3717   
       2023-11-30              67.9678                  21.3717   
       2023-12-31              67.9678                  21.3717   
       2024-01-31              67.9678                  21.3717   

                   EBITDA to revenue  EPS basic  EPS diluted  ...    PS TTM  \
Ticker Date                                                   ...             
ACU.A  2023-01-31           4.546930       0.86         0.82  ...  0.430501   
       2023-02-28           4.546930       0.86         0.82  ...  0.445459   
       2023-03-31           6.413892       0.28         0.28  ...  0.415088   
       2023-04-30           6.413892       0.28         0.28  ...  0.472839   
       2023-05-31           6.413892       0.28         0.28  ...  0.450822   
...                              ...        ...          ...  ...       ...   
ZYXI.O 2023-09-30           9.749192       0.24         0.23  ...  1.473115   
       2023-10-31           9.749192       0.24         0.23  ...  1.620008   
       2023-11-30           9.749192       0.24         0.23  ...  1.671089   
       2023-12-31           9.749192       0.24         0.23  ...  1.986699   
       2024-01-31           9.749192       0.24         0.23  ...  2.160011   

                      ROA      ROE      ROIC  Retained earnings per share  \
Ticker Date                                                                 
ACU.A  2023-01-31  3.9032   3.8400  2.418728                    20.057558   
       2023-02-28  3.9032   3.8400  2.418728                    20.057558   
       2023-03-31  1.3740   1.2357  0.791980             

In [6]:
# Choose the companies in the a specific industry
if industry == []:
    selected_industry = tickers
else:
    industries = pd.read_excel('industry_companies.xlsx', index_col=0)
    selected_industry = [x for x in industries.columns if any(y for y in industry if y in str(industries[x][0])) and x in tickers]

In [7]:
# Rank transform characteristics to [0,1]
period[characteristics] = period.groupby(['Date']).transform(lambda x: (1/(len(x)+1))*(np.argsort(np.argsort(np.array(x), kind='mergesort'), kind='mergesort')+1))[characteristics]
period

Alpha  Asset liability ratio      Beta  \
Ticker Date                                                    
ACU.A  2023-01-31  0.429709               0.449792  0.273546   
       2023-02-28  0.706371               0.450139  0.259349   
       2023-03-31  0.491343               0.406856  0.139889   
       2023-04-30  0.933518               0.406856  0.073753   
       2023-05-31  0.450485               0.406856  0.353186   
...                     ...                    ...       ...   
ZYXI.O 2023-09-30  0.910665               0.590028  0.871884   
       2023-10-31  0.936981               0.588643  0.603878   
       2023-11-30  0.387812               0.588643  0.615651   
       2023-12-31  0.710180               0.588296  0.740997   
       2024-01-31  0.867729               0.588296  0.584488   

                   CapEx to depreciation  Capital reserve per share  \
Ticker Date                                                           
ACU.A  2023-01-31               0.232687                   0.274584   
       2023-02-28               0.231648                   0.274931   
       2023-03-31               0.414820                   0.275970   
       2023-04-30               0.414127                   0.275623   
       2023-05-31               0.414820                   0.275623   
...                                  ...                        ...   
ZYXI.O 2023-09-30               0.781856                   0.188366   
       2023-10-31               0.783587                   0.188019   
       2023-11-30               0.782548                   0.187673   
       2023-12-31               0.780817                   0.187673   
       2024-01-31               0.780817                   0.187673   

                   Current asset ratio  Current liability ratio  \
Ticker Date                                                       
ACU.A  2023-01-31             0.729571                 0.259695   
       2023-02-28             0.730609                 0.260734   
       2023-03-31             0.722992                 0.357687   
       2023-04-30             0.722299                 0.358726   
       2023-05-31             0.722992                 0.358380   
...                                ...                      ...   
ZYXI.O 2023-09-30             0.776662                 0.197715   
       2023-10-31             0.777008                 0.197715   
       2023-11-30             0.776316                 0.196676   
       2023-12-31             0.776316                 0.195637   
       2024-01-31             0.776316                 0.195637   

                   EBITDA to revenue  EPS basic  EPS diluted  ...    PS TTM  \
Ticker Date                                                   ...             
ACU.A  2023-01-31           0.401662   0.611842     0.609418  ...  0.148199   
       2023-02-28           0.400970   0.612881     0.610457  ...  0.170360   
       2023-03-31           0.455679   0.614958     0.614612  ...  0.168283   
       2023-04-30           0.457410   0.618421     0.618075  ...  0.201524   
       2023-05-31           0.457410   0.617729     0.617382  ...  0.198753   
...                              ...        ...          ...  ...       ...   
ZYXI.O 2023-09-30           0.521468   0.532548     0.532895  ...  0.517659   
       2023-10-31           0.521814   0.533241     0.533241  ...  0.575831   
       2023-11-30           0.521468   0.531163     0.530817  ...  0.560596   
       2023-12-31           0.521468   0.530817     0.530471  ...  0.587950   
       2024-01-31           0.521468   0.530817     0.530471  ...  0.621884   

                        ROA       ROE      ROIC  Retained earnings per share  \
Ticker Date                                                                    
ACU.A  2023-01-31  0.555748  0.529778  0.529086                     0.811288   
       2023-02-28  0.554363  0.529778  0.529778                     0.811288   
       2023-03-31  0.568906  0.534626  0.5363

In [8]:
# Rolling accumulated returns in the given investment cycle without knowing the future returns
R = period[period.index.get_level_values(1).isin(dates[-T:])][['Return']]
R_rolling = R.groupby(level=0, group_keys=False).apply(lambda x: (x/100+1).rolling(window=I).apply(lambda y: y.prod())-1).dropna()
R_estimation = R_rolling[R_rolling.index.get_level_values(1).isin(dates[-T+I-1:])]
R_estimation.unstack(level=1).droplevel(0, axis=1)

Date,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
Ticker,,,,,,,
ACU.A,0.304114,0.222621,0.306879,0.291892,0.456105,0.726636,0.586995
AE.A,-0.270587,-0.365828,-0.111372,-0.153115,-0.175155,-0.243776,-0.262033
AGE.A,-0.026667,0.288136,0.025641,0.034483,-0.266000,-0.568869,-0.438356
AIM.A,0.196154,0.198077,0.058140,-0.039583,0.012931,-0.343284,-0.332797
AINC.A,-0.304734,-0.344098,-0.514578,-0.443269,-0.473953,-0.621399,-0.700000
...,...,...,...,...,...,...,...
ZKIN.O,-0.244172,-0.205674,0.026201,-0.290780,0.318841,0.340629,0.240260
ZLAB.O,-0.286664,-0.314401,-0.269092,-0.279588,-0.160049,-0.014425,-0.282435
ZS.O,0.291720,0.189859,0.331764,0.761265,0.458001,0.514422,0.469448


In [9]:
# De-mean returns
R_demean = R_estimation.groupby(level=1).transform(lambda x: x-x.mean(axis=0))
R_demean.unstack(level=1).droplevel(0, axis=1)

Date,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
Ticker,,,,,,,
ACU.A,0.294072,0.225681,0.330379,0.362523,0.447942,0.696557,0.636375
AE.A,-0.280628,-0.362769,-0.087873,-0.082484,-0.183318,-0.273855,-0.212653
AGE.A,-0.036708,0.291195,0.049140,0.105114,-0.274162,-0.598948,-0.388976
AIM.A,0.186112,0.201137,0.081639,0.031048,0.004769,-0.373363,-0.283418
AINC.A,-0.314775,-0.341038,-0.491079,-0.372638,-0.482115,-0.651479,-0.650620
...,...,...,...,...,...,...,...
ZKIN.O,-0.254214,-0.202614,0.049700,-0.220149,0.310678,0.310550,0.289639
ZLAB.O,-0.296705,-0.311341,-0.245593,-0.208957,-0.168211,-0.044504,-0.233055
ZS.O,0.281679,0.192919,0.355263,0.831896,0.449839,0.484343,0.518827


In [10]:
# Setting characteristics matrix
X = period[period.index.get_level_values(1).isin(dates[-1-T:])][characteristics]
X_shifted = X.groupby(['Ticker']).shift(I).dropna()
X_estimation = X_shifted[X_shifted.index.get_level_values(1).isin(dates[-1-T+I:])]
X_holding = X_shifted[X_shifted.index.get_level_values(1)==dates[-1]].droplevel(1)
X_estimation

Alpha  Asset liability ratio      Beta  \
Ticker Date                                                    
ACU.A  2023-07-31  0.429709               0.449792  0.273546   
       2023-08-31  0.706371               0.450139  0.259349   
       2023-09-30  0.491343               0.406856  0.139889   
       2023-10-31  0.933518               0.406856  0.073753   
       2023-11-30  0.450485               0.406856  0.353186   
...                     ...                    ...       ...   
ZYXI.O 2023-09-30  0.386427               0.304017  0.896122   
       2023-10-31  0.431094               0.305055  0.760042   
       2023-11-30  0.260388               0.305402  0.811981   
       2023-12-31  0.769044               0.563366  0.046745   
       2024-01-31  0.455679               0.563366  0.372230   

                   CapEx to depreciation  Capital reserve per share  \
Ticker Date                                                           
ACU.A  2023-07-31               0.232687                   0.274584   
       2023-08-31               0.231648                   0.274931   
       2023-09-30               0.414820                   0.275970   
       2023-10-31               0.414127                   0.275623   
       2023-11-30               0.414820                   0.275623   
...                                  ...                        ...   
ZYXI.O 2023-09-30               0.779086                   0.173476   
       2023-10-31               0.777355                   0.173823   
       2023-11-30               0.777355                   0.173823   
       2023-12-31               0.785665                   0.168975   
       2024-01-31               0.786357                   0.168975   

                   Current asset ratio  Current liability ratio  \
Ticker Date                                                       
ACU.A  2023-07-31             0.729571                 0.259695   
       2023-08-31             0.730609                 0.260734   
       2023-09-30             0.722992                 0.357687   
       2023-10-31             0.722299                 0.358726   
       2023-11-30             0.722992                 0.358380   
...                                ...                      ...   
ZYXI.O 2023-09-30             0.686288                 0.559903   
       2023-10-31             0.686981                 0.560596   
       2023-11-30             0.687327                 0.562327   
       2023-12-31             0.793629                 0.112188   
       2024-01-31             0.792936                 0.111842   

                   EBITDA to revenue  EPS basic  EPS diluted  ...       PEG  \
Ticker Date                                                   ...             
ACU.A  2023-07-31           0.401662   0.611842     0.609418  ...  0.313712   
       2023-08-31           0.400970   0.612881     0.610457  ...  0.306094   
       2023-09-30           0.455679   0.614958     0.614612  ...  0.306787   
       2023-10-31           0.457410   0.618421     0.618075  ...  0.294321   
       2023-11-30           0.457410   0.617729     0.617382  ...  0.296053   
...                              ...        ...          ...  ...       ...   
ZYXI.O 2023-09-30           0.426939   0.505886     0.509349  ...  0.003463   
       2023-10-31           0.430055   0.509349     0.512812  ...  0.003463   
       2023-11-30           0.429709   0.508657     0.512119  ...  0.004501   
       2023-12-31           0.492036   0.508657     0.511427  ...  0.004501   
       2024-01-31           0.490305   0.504501     0.507618  ...  0.004501   

                     PS TTM       ROA       ROE      ROIC  \
Ticker Date                                                 
ACU.A  2023-07-31  0.148199  0.555748  0.529778  0.529086   
       2023-08-31  0.170360  0.554363  0.529778  0.529778   
       2023-09-30  0.168283  0.568906  0.534626  0.536357   
       2023-10-31  0.201524  0.567521  0.536011  0.539474   

In [11]:
# Extracting the explained R
R_hat = X_estimation @ np.linalg.solve(X_estimation.T @ X_estimation, X_estimation.T @ R_demean)
R_hat = R_hat.unstack(level=1).droplevel(0,axis=1)
R_hat

Date,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
Ticker,,,,,,,
ACU.A,0.112079,0.106416,0.164077,0.145866,0.162950,0.159639,0.138748
AE.A,0.019496,0.028651,-0.008132,-0.015904,-0.010368,-0.104078,-0.097837
AGE.A,-0.238178,-0.221903,-0.355368,-0.356171,-0.337375,-0.376345,-0.360105
AIM.A,-0.097369,-0.083633,-0.071148,-0.091847,-0.079837,-0.106447,-0.088815
AINC.A,0.068563,0.068919,-0.039148,-0.041350,-0.043018,0.031317,0.022633
...,...,...,...,...,...,...,...
ZKIN.O,-0.055700,-0.035081,-0.046960,-0.041692,-0.045938,-0.034397,-0.034195
ZLAB.O,-0.074406,-0.042837,-0.004283,-0.007777,-0.008766,-0.019170,-0.025644
ZS.O,0.035633,0.020861,0.022518,0.029728,0.007511,0.026833,0.007217


In [12]:
# Eigendecomposition
RR_N = (R_hat @ R_hat.T) / N # Just like the expression of covariance between N stocks
eigen_values, eigen_vectors = eigsh(RR_N.to_numpy(), k=K, which='LM')
Gb_hat = np.sqrt(N) * pd.DataFrame(eigen_vectors, index=tickers, columns=[f'Factor {x}' for x in range(K, 0, -1)])
Gb_hat

# Calculation of the projection from X to G_beta
coef_beta = np.linalg.solve(X_estimation.T @ X_estimation, X_estimation.droplevel(1).T @ Gb_hat.loc[Gb_hat.index.repeat(T-I+1)])
coef_beta = pd.DataFrame(coef_beta @ eigen_values, index=characteristics, columns=[dates[-1]])
Gb_hat

,Factor 6,Factor 5,Factor 4,Factor 3,Factor 2,Factor 1
Ticker,,,,,,
ACU.A,-0.328447,1.538056,0.119286,-0.434069,1.133900,-1.771704
AE.A,0.946601,-0.467957,0.074082,-2.476120,-2.027201,0.337098
AGE.A,1.416514,-0.910757,1.035859,-0.065733,-3.304208,4.018604
AIM.A,0.932101,-0.425946,-1.006217,-0.502339,0.020335,1.108969
AINC.A,0.264777,0.831835,-0.134468,2.339593,-1.941651,-0.123310
...,...,...,...,...,...,...
ZKIN.O,1.347353,0.169672,-0.099608,0.302707,0.151437,0.526858
ZLAB.O,1.894501,0.856016,-0.413947,-0.639126,1.128779,0.328999
ZS.O,-1.544307,0.987998,-0.175368,-0.081265,-0.253395,-0.269679


In [13]:
# Using the original returns for data fit
R_estimation = R_estimation.unstack(level=1).droplevel(0, axis=1).loc[selected_industry]
R_estimation

Date,2023-07-31,2023-08-31,2023-09-30,2023-10-31,2023-11-30,2023-12-31,2024-01-31
Ticker,,,,,,,
AE.A,-0.270587,-0.365828,-0.111372,-0.153115,-0.175155,-0.243776,-0.262033
BRN.A,0.003860,0.120511,0.211379,-0.089025,-0.039936,-0.041715,-0.021997
CEI.A,-0.568528,-0.761628,-0.854430,-0.748571,-0.771963,-0.642633,-0.800000
ELLO.A,-0.067039,0.163200,0.293333,-0.239130,-0.217262,-0.169444,-0.060479
ENSV.A,-0.678740,-0.452542,-0.197895,-0.090909,-0.120787,-0.217391,-0.480392
...,...,...,...,...,...,...,...
VIA.O,-0.749972,-0.700000,-0.600109,-0.453140,-0.134981,0.350575,0.298654
VTNR.O,-0.276099,-0.552270,-0.549595,-0.454430,-0.477589,-0.457600,-0.722960
VVPR.O,-0.345494,-0.256866,-0.136364,-0.405714,-0.676439,-0.710210,-0.777049


In [14]:
theta = pd.DataFrame()
Gb_hat = Gb_hat.loc[selected_industry]
X_holding = X_holding.loc[selected_industry]
for i in range(T-I+1):

    # Return in the previous month
    R_previous = R_estimation[[dates[i-T+I-1]]]

    # Characteristics in the previous month
    X_previous = X_estimation[X_estimation.index.get_level_values(1)==dates[i-T+I-1]].droplevel(1,axis=0).loc[selected_industry]
    
    if method == 1:
        # First method - theta
        X_E = X_previous.to_numpy() - Gb_hat @ np.linalg.solve(Gb_hat.T @ Gb_hat, Gb_hat.T @ X_previous)
        X_E.columns = characteristics
        theta[dates[i-T+I-1]] = pd.DataFrame(np.linalg.solve(X_E.T @ X_E, X_E.T @ R_previous), index=characteristics)
    else:
        # Second method - theta
        # F_previous is the coefficient of Gb_hat with respect to R_previous, regardless of the effect of intercept Ga
        F_previous = pd.DataFrame(np.linalg.solve(Gb_hat.T @ Gb_hat, Gb_hat.T @ R_previous), index=Gb_hat.columns, columns=[dates[i-T+I-1]])
        # Final regression to derive the linear estimation of Ga on X, which is denoted as theta
        Y = R_previous - Gb_hat @ F_previous
        theta[dates[i-T+I-1]] = pd.DataFrame(np.linalg.solve(X_previous.T @ X_previous, X_previous.T @ Y), index=characteristics)

# Forcast the effect of characteristics by regressing in the estimation period
time_labels = pd.DataFrame([list(range(T-I+1))], index=['Order'], columns=[dates[-T+I-1:]]).T
holding_time_label = [[T]]
theta = pd.DataFrame(holding_time_label @ np.linalg.solve(time_labels.T @ time_labels, time_labels.T @ theta.T.values), index=[dates[-1]], columns=characteristics).T

if method == 1:
    # First method - G_alpha
    X_E_holding = X_holding.to_numpy() - Gb_hat @ np.linalg.solve(Gb_hat.T @ Gb_hat, Gb_hat.T @ X_holding)
    X_E_holding.columns = characteristics
    Ga_hat = X_E_holding @ theta
else:
    # Second method - G_alpha
    # Weight of my portfolio
    Ga_hat = X_holding @ theta
    
Ga_hat

,2024-01-31
Ticker,
AE.A,0.616065
BRN.A,-0.135207
CEI.A,-0.483793
ELLO.A,0.628595
ENSV.A,0.071583
...,...
VIA.O,-0.151896
VTNR.O,-0.125581
VVPR.O,-0.171010


In [15]:
# Scale towards a target vol
R_alpha = Ga_hat.T @ R_estimation
sd_scale_factor = 1 / (np.sqrt(T) * np.std(R_alpha, axis=1, ddof=1))
sd_scale_factor

2024-01-31    0.175426
dtype: float64

In [16]:
# Weight of my portfolio
omega = Ga_hat * sd_scale_factor
omega

,2024-01-31
Ticker,
AE.A,0.108074
BRN.A,-0.023719
CEI.A,-0.084870
ELLO.A,0.110272
ENSV.A,0.012558
...,...
VIA.O,-0.026646
VTNR.O,-0.022030
VVPR.O,-0.030000


In [17]:
# Absolute weight of the portfolio
omega['Abs'] = omega.iloc[:,0].abs()
omega = omega.sort_values(by='Abs',ascending=False).head(min(S, len(selected_industry))).drop('Abs', axis=1)
omega

,2024-01-31 00:00:00
Ticker,
HUSA.A,-0.189621
EP.A,-0.156489
SMR.N,-0.148417
CEPU.N,0.128284
BORR.N,-0.126083
ELLO.A,0.110272
AE.A,0.108074
TNP.N,0.105244
TUSK.O,0.100570
